# Standarisation du process d'ingestions des fichiers de prévisions mensuelles pour les fichiers excel à feuille unique

In [220]:
%config Completer.use_jedi = False

In [230]:
import sys 
sys.path.append('/home/cdsw/code')
from utils import *
import numpy as np
import math
import openpyxl
import pandas as pd
import dateutil
from dateutil.parser import parse
from Prev.single_sheet_process import *
import warnings
import imp
warnings.filterwarnings('ignore')


In [232]:
import Prev.multiple_sheet_process
import utils
imp.reload(utils)
imp.reload(Prev.multiple_sheet_process)
from utils import *
from Prev.multiple_sheet_process import * 

In [233]:
pd.set_option("max_rows", 5000)

In [234]:
emplacement = "/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions DEA mensualisées"
extension = "*.xlsx"
list_filename = get_files_in_directory(emplacement,extension)
filename = list_filename[-8]
print(filename)

/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions DEA mensualisées/ajout_plateforme/Equilibre Technique mensualisé 2021 10 - DEA.xlsx


In [235]:
df = pd.read_excel(filename, engine ='openpyxl')

In [236]:
df

,Unnamed: 0,2020,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62
0,NaN,janvier,février,mars,avril,mai,juin,juillet,août,septembre,...,2022.000000,2023.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,En millions d'euros,Observé,Observé,Observé,Observé,Observé,Observé,Observé,Observé,Observé,...,2022.000000,2023.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Contributions d'Assurance chômage,3655.35,3124.23,2538.97,2180.77,1887.6,2478.48,3366.74,3201.03,3066.1,...,41122.564213,41976.536879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Contributions principales,2395.8,1939.01,1645.39,1408.2,1174.2,1591.13,2137.43,2035.76,1919.82,...,25921.911809,26427.117482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Impositions de toutes natures,1259.55,1185.23,893.577,772.565,713.407,887.352,1229.31,1165.27,1146.27,...,15200.652404,15549.419397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Conventions diverses, y compris UE",32.0043,0.714397,3.06734,3.49915,0.353575,-0.190282,5.00901,-2.978,1.79045,...,300.000000,200.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Participation entreprise CSP / CRP,34.6258,40.0985,35.1404,21.5838,29.2888,37.3818,26.5881,21.7101,37.3209,...,565.156140,398.730803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Autres produits,0,0,0,0,0,0,0,0,1,...,78.000000,78.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Total recettes,3721.98,3165.05,2577.17,2205.85,1917.25,2515.67,3398.33,3219.76,3105.21,...,41987.720352,42575.267682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [237]:
# récupérer le nombre de feuilles dans le fichier excel
#openpyxl.load_workbook(filename).sheetnames
#res = len(openpyxl.load_workbook(filename).sheetnames)
#df =pd.read_excel(filename, sheet_name = 7, engine = 'openpyxl')


## Travail sur les données 

In [238]:
df = pd.read_excel(filename,engine ='openpyxl')

In [239]:
stop_l, stop_c, column_bloc = check_index(df)
if stop_c[0] != column_bloc: 
    df.loc[df[df.columns[stop_c[0]]].isna(), df.columns[stop_c[0]]] = \
    df[df[df.columns[stop_c[0]]].isna()][df.columns[column_bloc]]
    
    
df =df[df.columns[stop_c[0]:stop_c[1]]]
df =df[:stop_l]

In [240]:
df = df.dropna(axis = 0, how = 'all')
df

,Unnamed: 0,2020,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
0,NaN,janvier,février,mars,avril,mai,juin,juillet,août,septembre,...,mars,avril,mai,juin,juillet,août,septembre,octobre,novembre,décembre
1,En millions d'euros,Observé,Observé,Observé,Observé,Observé,Observé,Observé,Observé,Observé,...,Prévu,Prévu,Prévu,Prévu,Prévu,Prévu,Prévu,Prévu,Prévu,Prévu
2,Contributions d'Assurance chômage,3655.35,3124.23,2538.97,2180.77,1887.6,2478.48,3366.74,3201.03,3066.1,...,3112.61,3572.04,3582.14,3272.35,3906.79,3603.86,3232.12,3573.51,3418.22,3605.76
3,Contributions principales,2395.8,1939.01,1645.39,1408.2,1174.2,1591.13,2137.43,2035.76,1919.82,...,1959.53,2308.45,2215.65,2039.24,2465.82,2232.44,2036.99,2263.22,2045.6,2293.61
4,Impositions de toutes natures,1259.55,1185.23,893.577,772.565,713.407,887.352,1229.31,1165.27,1146.27,...,1153.08,1263.58,1366.49,1233.11,1440.98,1371.42,1195.13,1310.29,1372.62,1312.15
5,"Conventions diverses, y compris UE",32.0043,0.714397,3.06734,3.49915,0.353575,-0.190282,5.00901,-2.978,1.79045,...,16.6667,16.6667,16.6667,16.6667,16.6667,16.6667,16.6667,16.6667,16.6667,16.6667
6,Participation entreprise CSP / CRP,34.6258,40.0985,35.1404,21.5838,29.2888,37.3818,26.5881,21.7101,37.3209,...,43.4075,23.6053,22.8877,31.3576,45.4002,55.0836,38.3854,25.6025,20.3912,29.2561
7,Autres produits,0,0,0,0,0,0,0,0,1,...,3,4,5,6,7,8,9,10,11,12
8,Total recettes,3721.98,3165.05,2577.17,2205.85,1917.25,2515.67,3398.33,3219.76,3105.21,...,3172.68,3612.31,3621.69,3320.37,3968.86,3675.61,3287.17,3615.78,3455.28,3651.68
10,Dépenses allocataires,3800.45,3023.15,2820.68,3910.08,3479.77,3688.81,4125.83,3039.94,3196.13,...,2638.85,3548.35,2590.11,2636.44,3284.37,2563.25,2724.31,3368.53,2690.33,2683.58


In [241]:
df = df.T
df = df.reset_index(level=0)
df = df.rename(columns = {"index" : "annee"})
df

,annee,0,1,2,3,4,5,6,7,8,...,32,33,34,35,36,37,39,41,42,43
0,Unnamed: 0,NaN,En millions d'euros,Contributions d'Assurance chômage,Contributions principales,Impositions de toutes natures,"Conventions diverses, y compris UE",Participation entreprise CSP / CRP,Autres produits,Total recettes,...,Financement et frais de gestion opérateurs,11% Pôle Emploi,"Autres (accompagnement CSP, recouvrements)",Frais de fonctionnement et de gestion,Intérêts sur emprunts,Total Dépenses,Solde,NaN,Solde financier,Dette
1,2020,janvier,Observé,3655.35,2395.8,1259.55,32.0043,34.6258,0,3721.98,...,355.022,345.372,9.65005,2.91667,-1.50063,4168.89,-446.913,0,-446.913,-37261.9
2,Unnamed: 2,février,Observé,3124.23,1939.01,1185.23,0.714397,40.0985,0,3165.05,...,350.652,345.372,5.27981,2.91667,16.5452,3397.54,-232.491,0,-232.491,-37494.4
3,Unnamed: 3,mars,Observé,2538.97,1645.39,893.577,3.06734,35.1404,0,2577.17,...,350.656,345.372,5.28415,2.91667,6.97805,4181.23,-1604.06,0,-1604.06,-39098.5
4,Unnamed: 4,avril,Observé,2180.77,1408.2,772.565,3.49915,21.5838,0,2205.85,...,345.447,345.372,0.0747107,2.91667,112.008,7320.45,-5114.6,0,-5114.6,-44213.1
5,Unnamed: 5,mai,Observé,1887.6,1174.2,713.407,0.353575,29.2888,0,1917.25,...,345.43,345.372,0.057557,2.91667,125.244,2733.55,-816.308,0,-816.308,-45029.4
6,Unnamed: 6,juin,Observé,2478.48,1591.13,887.352,-0.190282,37.3818,0,2515.67,...,345.442,345.372,0.0694583,2.91667,7.57262,5633.8,-3118.13,0,-3118.13,-48147.5
7,Unnamed: 7,juillet,Observé,3366.74,2137.43,1229.31,5.00901,26.5881,0,3398.33,...,345.462,345.372,0.0898929,2.91667,14.6961,5563.2,-2164.86,0,-2164.86,-50312.4
8,Unnamed: 8,août,Observé,3201.03,2035.76,1165.27,-2.978,21.7101,0,3219.76,...,345.461,345.372,0.0886666,2.91667,-4.21632,3855.57,-635.805,0,-635.805,-50948.2
9,Unnamed: 9,septembre,Observé,3066.1,1919.82,1146.27,1.79045,37.3209,1,3105.21,...,345.457,345.372,0.0844531,2.91667,-0.686814,3903.8,-798.591,0,-798.591,-51746.8


In [242]:
# remplir la colonne année 
df = df.rename(columns = {'index' : 'annee'})
year_values= df[df[0] == 'janvier']['annee'].tolist()
j= 0
for i in range(2,len(df)):
    if df.iloc[i][0] != 'janvier':
        df.iloc[i]['annee'] = year_values[j]
    else :
        j+= 1
        df.iloc[i]['annee'] = year_values[j]



In [243]:
df1= df[df.columns[:3]]

# changer l'entete de la table

df1= df1[1:].reset_index().drop(columns = 'index')
df1 = df1.rename(columns = {0:'mois_nom', 1:'type_montant'})

df1['key'] = [i for i in range(len(df1))]

df1

,annee,mois_nom,type_montant,key
0,2020,janvier,Observé,0
1,2020,février,Observé,1
2,2020,mars,Observé,2
3,2020,avril,Observé,3
4,2020,mai,Observé,4
5,2020,juin,Observé,5
6,2020,juillet,Observé,6
7,2020,août,Observé,7
8,2020,septembre,Observé,8
9,2020,octobre,Observé,9


In [73]:
df2= df[df.columns[3:]]
df2

,2,3,4,5,6,7,8,10,11,12,...,32,33,34,35,36,37,39,41,42,43
0,Contributions d'Assurance chômage,Contributions principales,Impositions de toutes natures,"Conventions diverses, y compris UE",Participation entreprise CSP / CRP,Autres produits,Total recettes,Dépenses allocataires,Allocations brutes,ARE / AREF,...,Financement et frais de gestion opérateurs,11% Pôle Emploi,"Autres (accompagnement CSP, recouvrements)",Frais de fonctionnement et de gestion,Intérêts sur emprunts,Total Dépenses,Solde,NaN,Solde financier,Dette
1,3655.35,2395.8,1259.55,32.0043,34.6258,0,3721.98,3800.45,3055.95,3062.05,...,355.022,345.372,9.65005,2.91667,-1.50063,4168.89,-446.913,0,-446.913,-37261.9
2,3124.23,1939.01,1185.23,0.714397,40.0985,0,3165.05,3023.15,3058.66,3056.66,...,350.652,345.372,5.27981,2.91667,16.5452,3397.54,-232.491,0,-232.491,-37494.4
3,2538.97,1645.39,893.577,3.06734,35.1404,0,2577.17,2820.68,2836.98,2829.57,...,350.656,345.372,5.28415,2.91667,6.97805,4181.23,-1604.06,0,-1604.06,-39098.5
4,2180.77,1408.2,772.565,3.49915,21.5838,0,2205.85,3910.08,3215.45,3184.33,...,345.447,345.372,0.0747107,2.91667,112.008,7320.45,-5114.6,0,-5114.6,-44213.1
5,1887.6,1174.2,713.407,0.353575,29.2888,0,1917.25,3479.77,3559.6,3524.68,...,345.43,345.372,0.057557,2.91667,125.244,2733.55,-816.308,0,-816.308,-45029.4
6,2478.48,1591.13,887.352,-0.190282,37.3818,0,2515.67,3688.81,3757.42,3741.03,...,345.442,345.372,0.0694583,2.91667,7.57262,5633.8,-3118.13,0,-3118.13,-48147.5
7,3366.74,2137.43,1229.31,5.00901,26.5881,0,3398.33,4125.83,3243.49,3259.74,...,345.462,345.372,0.0898929,2.91667,14.6961,5563.2,-2164.86,0,-2164.86,-50312.4
8,3201.03,2035.76,1165.27,-2.978,21.7101,0,3219.76,3039.94,3094.31,3100.39,...,345.461,345.372,0.0886666,2.91667,-4.21632,3855.57,-635.805,0,-635.805,-50948.2
9,3066.1,1919.82,1146.27,1.79045,37.3209,1,3105.21,3196.13,3246.06,3243.76,...,345.457,345.372,0.0844531,2.91667,-0.686814,3903.8,-798.591,0,-798.591,-51746.8


In [74]:
df2= df[df.columns[3:]]

new_header = df2.iloc[0] #grab the first row for the header
df2 = df2[1:] #take the data less the header row
df2.columns = new_header #set the header row as the df header

df2

,Contributions d'Assurance chômage,Contributions principales,Impositions de toutes natures,"Conventions diverses, y compris UE",Participation entreprise CSP / CRP,Autres produits,Total recettes,Dépenses allocataires,Allocations brutes,ARE / AREF,...,Financement et frais de gestion opérateurs,11% Pôle Emploi,"Autres (accompagnement CSP, recouvrements)",Frais de fonctionnement et de gestion,Intérêts sur emprunts,Total Dépenses,Solde,NaN,Solde financier,Dette
1,3655.35,2395.8,1259.55,32.0043,34.6258,0,3721.98,3800.45,3055.95,3062.05,...,355.022,345.372,9.65005,2.91667,-1.50063,4168.89,-446.913,0,-446.913,-37261.9
2,3124.23,1939.01,1185.23,0.714397,40.0985,0,3165.05,3023.15,3058.66,3056.66,...,350.652,345.372,5.27981,2.91667,16.5452,3397.54,-232.491,0,-232.491,-37494.4
3,2538.97,1645.39,893.577,3.06734,35.1404,0,2577.17,2820.68,2836.98,2829.57,...,350.656,345.372,5.28415,2.91667,6.97805,4181.23,-1604.06,0,-1604.06,-39098.5
4,2180.77,1408.2,772.565,3.49915,21.5838,0,2205.85,3910.08,3215.45,3184.33,...,345.447,345.372,0.0747107,2.91667,112.008,7320.45,-5114.6,0,-5114.6,-44213.1
5,1887.6,1174.2,713.407,0.353575,29.2888,0,1917.25,3479.77,3559.6,3524.68,...,345.43,345.372,0.057557,2.91667,125.244,2733.55,-816.308,0,-816.308,-45029.4
6,2478.48,1591.13,887.352,-0.190282,37.3818,0,2515.67,3688.81,3757.42,3741.03,...,345.442,345.372,0.0694583,2.91667,7.57262,5633.8,-3118.13,0,-3118.13,-48147.5
7,3366.74,2137.43,1229.31,5.00901,26.5881,0,3398.33,4125.83,3243.49,3259.74,...,345.462,345.372,0.0898929,2.91667,14.6961,5563.2,-2164.86,0,-2164.86,-50312.4
8,3201.03,2035.76,1165.27,-2.978,21.7101,0,3219.76,3039.94,3094.31,3100.39,...,345.461,345.372,0.0886666,2.91667,-4.21632,3855.57,-635.805,0,-635.805,-50948.2
9,3066.1,1919.82,1146.27,1.79045,37.3209,1,3105.21,3196.13,3246.06,3243.76,...,345.457,345.372,0.0844531,2.91667,-0.686814,3903.8,-798.591,0,-798.591,-51746.8
10,3162.67,1889.7,1272.96,1.69444,31.5523,0,3195.91,3875.38,2993.6,2977.7,...,345.462,345.372,0.0902039,2.91667,-5.7588,4767.91,-1572,0,-1572,-53318.8


In [75]:

df3= df2.T.unstack().reset_index(level=1, name='montant_millions')
df3= df3.rename(columns ={0:'nature_montant'})

l = [len(df2.columns)*[i] for i in range(len(df1))]
flat_l = [item for sublist in l for item in sublist]

df3['key'] = flat_l



In [76]:
df3

,nature_montant,montant_millions,key
1,Contributions d'Assurance chômage,3655.35,0
1,Contributions principales,2395.8,0
1,Impositions de toutes natures,1259.55,0
1,"Conventions diverses, y compris UE",32.0043,0
1,Participation entreprise CSP / CRP,34.6258,0
1,Autres produits,0,0
1,Total recettes,3721.98,0
1,Dépenses allocataires,3800.45,0
1,Allocations brutes,3055.95,0
1,ARE / AREF,3062.05,0


In [77]:
dff = df3.merge(df1, on= 'key')
dff.drop(columns ='key', inplace = True)
dff['montant_millions'] = dff['montant_millions'].astype(float, errors = 'ignore')


In [78]:
mapping_month = {'janvier': 1, 'février':2 , 'mars': 3, 'avril': 4, 'mai':5, 'juin':6 , 'juillet':7 , 'août' : 8 ,
                 'septembre' :9 , 'octobre' : 10 , 'novembre' : 11 , 'décembre' : 12 }

dff['mois'] = dff['mois_nom'].map(mapping_month)

In [79]:
dff

,nature_montant,montant_millions,annee,mois_nom,type_montant,mois
0,Contributions d'Assurance chômage,3655.348133,2020,janvier,Observé,1
1,Contributions principales,2395.795352,2020,janvier,Observé,1
2,Impositions de toutes natures,1259.552781,2020,janvier,Observé,1
3,"Conventions diverses, y compris UE",32.004273,2020,janvier,Observé,1
4,Participation entreprise CSP / CRP,34.625761,2020,janvier,Observé,1
5,Autres produits,0.000000,2020,janvier,Observé,1
6,Total recettes,3721.978167,2020,janvier,Observé,1
7,Dépenses allocataires,3800.452507,2020,janvier,Observé,1
8,Allocations brutes,3055.949747,2020,janvier,Observé,1
9,ARE / AREF,3062.045231,2020,janvier,Observé,1


In [80]:
version_prev = get_date_extraction_m_yy(filename)
dff['version_prev'] = version_prev
dff['version_prev'] = pd.to_datetime(dff['version_prev'])
dff['version_prev']= dff['version_prev'].dt.strftime('%Y-%m')

In [81]:
dff = dff[['version_prev','annee' , 'mois' , 'mois_nom', 'nature_montant',  'type_montant', 'montant_millions' ]]
    
dff

,version_prev,annee,mois,mois_nom,nature_montant,type_montant,montant_millions
0,2021-10,2020,1,janvier,Contributions d'Assurance chômage,Observé,3655.348133
1,2021-10,2020,1,janvier,Contributions principales,Observé,2395.795352
2,2021-10,2020,1,janvier,Impositions de toutes natures,Observé,1259.552781
3,2021-10,2020,1,janvier,"Conventions diverses, y compris UE",Observé,32.004273
4,2021-10,2020,1,janvier,Participation entreprise CSP / CRP,Observé,34.625761
5,2021-10,2020,1,janvier,Autres produits,Observé,0.000000
6,2021-10,2020,1,janvier,Total recettes,Observé,3721.978167
7,2021-10,2020,1,janvier,Dépenses allocataires,Observé,3800.452507
8,2021-10,2020,1,janvier,Allocations brutes,Observé,3055.949747
9,2021-10,2020,1,janvier,ARE / AREF,Observé,3062.045231


# Code sous forme de fonction

In [7]:
prevision_data_ingestion_single_sheet(filename, extension)

durée de traitement = 0:00:00.183093


,version_prev,annee,mois,mois_nom,nature_montant,type_montant,montant_millions
0,2021-02,2020,1,janvier,Contributions d'Assurance chômage,Observé,3655.348133
1,2021-02,2020,1,janvier,Contributions principales,Observé,2395.795352
2,2021-02,2020,1,janvier,Impositions de toutes natures,Observé,1259.552781
3,2021-02,2020,1,janvier,"Conventions diverses, y compris UE",Observé,32.004273
4,2021-02,2020,1,janvier,Participation entreprise CSP / CRP,Observé,34.625761
5,2021-02,2020,1,janvier,Autres produits,Observé,0.000000
6,2021-02,2020,1,janvier,Total recettes,Observé,3721.978167
7,2021-02,2020,1,janvier,Dépenses allocataires,Observé,3800.452507
8,2021-02,2020,1,janvier,Allocations brutes,Observé,3055.949747
9,2021-02,2020,1,janvier,ARE / AREF,Observé,3062.045231
